In [2]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

In [54]:
class DataSet(object):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)
        self.row_num = len(self.data)
        self.col_num = len(self.data.columns)
        #self.missing_value_process()

        #self.raw_predictor = self.data.iloc[:,:-1].values
        #self.raw_response = self.data.iloc[:,-1].values

        self.missing_value_process()
    
    def missing_value_process(self):
        self.processed_data = self.data
        for col in range(self.col_num):
            qm_num = 0
            for row in range(self.row_num):
                if str(self.data.iat[row,col]) == "?":
                    qm_num += 1
            if qm_num/self.row_num >= 0.5:
                self.processed_data.drop(self.processed_data.columns[[col]], axis=1)

            

        #self.processed_data = self.raw_data.dropna()





In [53]:
train_data = DataSet('diabetic_data_training.csv')
test_data = DataSet('diabetic_data_test.csv')

#train_data.data
train_data.processed_data


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
1,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
2,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
3,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
4,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91584,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
91585,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
91586,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
91587,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [8]:
train_data = pd.read_csv('diabetic_data_training.csv')
test_data = pd.read_csv('diabetic_data_test.csv')

predictor = train_data.iloc[:,:-1].values
response = train_data.iloc[:,-1].values

predictor,response 

(array([[149190, 55629189, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [64410, 86047875, 'AfricanAmerican', ..., 'No', 'No', 'Yes'],
        [500364, 82442376, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        ...,
        [443854148, 41088789, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [443857166, 31693671, 'Caucasian', ..., 'No', 'Ch', 'Yes'],
        [443867222, 175429310, 'Caucasian', ..., 'No', 'No', 'No']],
       dtype=object),
 array(['>30', 'NO', 'NO', ..., 'NO', 'NO', 'NO'], dtype=object))

In [5]:
qm = 0 # Question Mark
total = 0 # Total 

for row in train_data:  

    if "?" in str(row['weight']): 

        qm += 1 

    total += 1 

if (qm / total) > 0.5: # Number of question mark greater than 50% 

    print("delete", qm, total) 

TypeError: string indices must be integers, not 'str'

In [ ]:
import csv  

# Open file  

with open('trainning.csv') as file_obj:  

       

    # Create reader object by passing the file  

    # object to DictReader method  

    reader_obj = csv.DictReader(file_obj)  

       

    # Iterate over each row in the csv file  

    # using reader object  

    qm = 0 # Question Mark 

    total = 0 # Total 

    for row in reader_obj:  

        if "?" in str(row['weight']): 

            qm += 1 

        total += 1 

    if (qm / total) > 0.5: # Number of question mark greater than 50% 

        print("delete", qm, total) 